In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import h5py
from matplotlib import animation
from matplotlib import cm
from mpl_toolkits import mplot3d
import os,sys
sys.path.append('/Users/ajc/Core/PyScript')
import readdata as rd
import seaborn as sns

In [2]:
path_root='/Users/ajc/FUSE/simulations/production/streams/'
#path_root='/Users/ajc/Core/simulations/production/streams/'
path = path_root

In [69]:
dir_name=['SIM_14']
path = [os.path.join(path_root,d) for d in dir_name]

sim_no =  500
SimNBGC = [os.path.join(path[0],'S3_NBGC_dx%sm_min250m'%sim_no), os.path.join(path[0],'S1_NBGC_dx%sm_min250m'%sim_no),
            os.path.join(path[0],'S2_NBGC_dx%sm_min250m'%sim_no)]
SimBGC = [os.path.join(path[0],'S3_BGC_dx%sm_min250m'%sim_no), os.path.join(path[0],'S1_BGC_dx%sm_min250m'%sim_no),
            os.path.join(path[0],'S2_BGC_dx%sm_min250m'%sim_no)]

                
#SimNBGC = [os.path.join(path[0],'Srun1')]
#SimBGC = [os.path.join(path[0],'Srun2')]
print (path)

['/Users/ajc/FUSE/simulations/production/streams/SIM_14']


In [70]:
Simulations = np.concatenate([SimNBGC, SimBGC])
Colors = ['r','k','b','g','gray']
#Sim_dis = [r'NBGC ($\alpha$=1E-4)',r'NBGC ($\alpha$=2.5E-4)',r'BGC ($\alpha$=1E-4)',r"BGC ($\alpha$=2.5E-4)"]
#Sim_dis = ['NBGC', r'NBGC ($O_2\updownarrow$)','BGC',r'BGC ($O_2\updownarrow$)']
#Sim_dis = ['NBGC', 'BGC']
Sim_dis = ['noBGC_LE', 'noBGC_ME', 'noBGC_HE', 'BGC_LE', 'BGC_ME', 'BGC_HE']
Simulations

array(['/Users/ajc/FUSE/simulations/production/streams/SIM_14/S3_NBGC_dx500m_min250m',
       '/Users/ajc/FUSE/simulations/production/streams/SIM_14/S1_NBGC_dx500m_min250m',
       '/Users/ajc/FUSE/simulations/production/streams/SIM_14/S2_NBGC_dx500m_min250m',
       '/Users/ajc/FUSE/simulations/production/streams/SIM_14/S3_BGC_dx500m_min250m',
       '/Users/ajc/FUSE/simulations/production/streams/SIM_14/S1_BGC_dx500m_min250m',
       '/Users/ajc/FUSE/simulations/production/streams/SIM_14/S2_BGC_dx500m_min250m'],
      dtype='<U76')

In [71]:
components = [(1,'DOM1'), (3,'NO3-'),(2,'O2(aq)')]
#components = [(1,'DOM1'), (3,'NO3-'),(2,'O2(aq)'), (4,'HCO3-'),(5,'H+'), (6,'N2(aq)')]

marks = ['solid','dashed', 'dotted', 'dashdot']

In [72]:
def get_perc_reduction(d_init,d_):
    data_pc = [100*(dat[0][0] - d1 )/dat[0][0] for d1 in data] # Percentage reduction
surf_files = ['surface_outlet_stream_1_2.dat', 
              'surface_outlet_stream_111_112.dat', 'surface_outlet_stream_198_199.dat',
              'surface_outlet_stream_221_222.dat', 'surface_outlet_stream_24_44.dat', 
              'surface_outlet_stream_260_262.dat','surface_outlet_stream_74_75.dat']

surf_loc = ['loc_1_2', 'loc_111_112', 'loc_198_199',
              'loc_221_222', 'loc_24_44',  'loc_260_262','loc_74_75']

In [82]:
#plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
%matplotlib qt
outfile='/Users/ajc/Core/PostProcessing/ReactiveTransport/2021/Q2/PR/BTC/'
yLims = [[-10, 400], [0,200], [0,300], [0,200], [0,200], [26000,27000]]
#yLims = [[-10, 650], [0,320], [0,350], [0,200], [0,200], [26000,27000]]
#yLims = [[-10, 1010], [0,510], [0,350], [0,200], [0,200], [26000,27000]]
def get_discharge_fromfile(comp=0):
    fig, axs = plt.subplots(1,3, figsize=(8,3), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1,1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    plt.tight_layout()
    axs = axs.ravel()
    to_micro = 1e+6
    for i,sim in enumerate(Simulations):

        for j,Tc in enumerate(components):
            print (sim,Tc)
            infile = os.path.join(sim,surf_files[comp])
            
            time = np.loadtxt(infile,usecols=0,skiprows=11)/(86400.)
            data = np.loadtxt(infile,usecols=Tc[0],skiprows=11)
            #print (data[:5])
            if 'NBGC' in sim:
                axs[j].plot(time,1e-3*data*to_micro,color=Colors[i], linestyle=marks[0], label = Sim_dis[i],linewidth=1.)
            else:
                axs[j].plot(time,1e-3*data*to_micro,color=Colors[i-3], linestyle=marks[1], label = Sim_dis[i], linewidth=1.)

            axs[j].set_xlabel('Time [d]', fontsize=12,fontweight='normal')
            
            axs[j].set_ylim(yLims[j][0],yLims[j][1])
            axs[j].set_title(Tc[1])
            print (data[-1])
            #axs[j].set_yticklabel_format(style='sci', axis='y', scilimits=(0,0),useMathText=True)
            if j ==0:
                axs[j].legend(loc='center right',fontsize=5)
                axs[j].set_ylabel('Concentration [$\mu$M]', fontsize=12,fontweight='normal')
            
    plt.tight_layout()
    #plt.savefig(outfile + '%s_%s_dtmean5.png'%(dir_name[0],dir_name[1]),dpi=200)
    plt.savefig(outfile + 'BTC-S1_S2_S3_dx%sm_min250m_%s.png'%(sim_no, surf_loc[comp]),dpi=400)



In [83]:
get_discharge_fromfile(comp=0)

/Users/ajc/FUSE/simulations/production/streams/SIM_14/S3_NBGC_dx500m_min250m (1, 'DOM1')
0.3522227040376
/Users/ajc/FUSE/simulations/production/streams/SIM_14/S3_NBGC_dx500m_min250m (3, 'NO3-')
0.1761113520188
/Users/ajc/FUSE/simulations/production/streams/SIM_14/S3_NBGC_dx500m_min250m (2, 'O2(aq)')
0.2863355668996
/Users/ajc/FUSE/simulations/production/streams/SIM_14/S1_NBGC_dx500m_min250m (1, 'DOM1')
0.3519356492824
/Users/ajc/FUSE/simulations/production/streams/SIM_14/S1_NBGC_dx500m_min250m (3, 'NO3-')
0.1759678246412
/Users/ajc/FUSE/simulations/production/streams/SIM_14/S1_NBGC_dx500m_min250m (2, 'O2(aq)')
0.2863354462155
/Users/ajc/FUSE/simulations/production/streams/SIM_14/S2_NBGC_dx500m_min250m (1, 'DOM1')
0.3513623860809
/Users/ajc/FUSE/simulations/production/streams/SIM_14/S2_NBGC_dx500m_min250m (3, 'NO3-')
0.1756811930405
/Users/ajc/FUSE/simulations/production/streams/SIM_14/S2_NBGC_dx500m_min250m (2, 'O2(aq)')
0.2863350085484
/Users/ajc/FUSE/simulations/production/streams/SI

In [ ]:
#plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
%matplotlib qt
outfile='/Users/ajc/Core/PostProcessing/ReactiveTransport/2021/Q2/'
yLims = [[-10, 600], [0,180], [100,350], [0,200], [0,200], [26000,27000]]
SimNBGC = [os.path.join(path[0],'FF_NBGC_S1_dx10000m')]
SimBGC = [os.path.join(path[0],'FF_BGC_S1_dx10000m')]
Simulations_A = np.concatenate([SimNBGC, SimBGC])
surf_files = ['surface_outlet_stream_198_199.dat', 'surface_outlet_stream_260_262.dat',
              'surface_outlet_stream_111_112.dat', 'surface_outlet_stream_74_75.dat']
def get_discharge_fromfile_all(comp=1):
    #fig, axs = plt.subplots(1,2, figsize=(8,3), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1]})
    #fig.subplots_adjust(hspace =.02, wspace=1.02)
    #plt.tight_layout()
    #axs = axs.ravel()
    
    to_micro = 1e+6
    p_count = 1
    for d_f in surf_files:
        for i,sim in enumerate(Simulations_A):
            axs = plt.subplot(4, 2,p_count)
            for j,Tc in enumerate(components[:2]):
                #print (sim,Tc)
                infile = os.path.join(sim,d_f)

                time = np.loadtxt(infile,usecols=0,skiprows=11)/(86400.)
                data = np.loadtxt(infile,usecols=Tc[0],skiprows=11)
                #print (data[:5])
                if 'NBGC' in sim:
                    axs.plot(time,1e-3*data*to_micro,color=Colors[i], linestyle=marks[0], label = Sim_dis[i])
                else:
                    axs.plot(time,1e-3*data*to_micro,color=Colors[i-2], linestyle=marks[1], label = Sim_dis[i])

                #axs.set_xlabel('Time [d]', fontsize=12,fontweight='normal')
                #axs.set_ylabel('Concentration [$\mu$M]', fontsize=12,fontweight='normal')
                #axs.set_ylim(yLims[j][0],yLims[j][1])
                #axs.set_title(Tc[1])
                #axs[j].set_yticklabel_format(style='sci', axis='y', scilimits=(0,0),useMathText=True)
                if j ==0:
                    axs.legend(loc='upper left',fontsize=7)
            p_count = p_count + 1
    plt.tight_layout()
    #plt.savefig(outfile + '%s_%s_dtmean5.png'%(dir_name[0],dir_name[1]),dpi=200)
    #plt.savefig(outfile + 'BTC-S2_min250m.png',dpi=200)




In [ ]:
get_discharge_fromfile_all()